# Import package

In [1]:
import numpy as np  
import pandas as pd 
import sys          
import random       

# Get training data

In [2]:
X_full = pd.read_csv('./input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
# choose last 10 data as test data                                                         
X_test_full = X_full.iloc[-10:,:]                                        
X_full = X_full.iloc[:-10,:]                                             

# Features Selection
Only sellect numerical features. 

In [3]:
# Select numerical features only                       
X_full = X_full.select_dtypes(exclude='object')        
                                                       
# Drop features which corr value is less than 0.4      
X_full = X_full.loc[:,X_full.corr()['SalePrice'] > 0.4]
                                                       
# Drop feature GarageArea because it is almost as same as GarageCars
features_to_be_droped = ['GarageArea']                 
X_full = X_full.drop(features_to_be_droped, axis=1)    
                                                       
print("Features after drop:")                          
for feature_name in X_full:                            
    print(feature_name)                                

Features after drop:
OverallQual
YearBuilt
YearRemodAdd
MasVnrArea
TotalBsmtSF
1stFlrSF
GrLivArea
FullBath
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
SalePrice


# Data Missing
Dealing with data missing

In [4]:
# Number of missing values in each column of training data                
data_missing_cnt = X_full.isnull().sum()                                  
print(data_missing_cnt[data_missing_cnt > 0].sort_values(ascending=False))

GarageYrBlt    79
MasVnrArea      8
dtype: int64


In [5]:
# Drop feature 'GarageYrBlt' because there is a lot of missing value
X_full = X_full.drop('GarageYrBlt', axis=1)            
                                                       
# Remove observations which 'MasVnrArea' is null       
X_full = X_full.dropna(axis = 0, subset=['MasVnrArea'])

# Get the number of entries with missing values in each column
data_missing_cnt = X_full.isnull().sum()
print(data_missing_cnt[data_missing_cnt > 0].sort_values(ascending=False))

In [6]:
sellected_features = ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 
                      'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 
                      'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'SalePrice']
X_test_full = X_test_full[sellected_features]
X_test_full = X_test_full.dropna(axis = 0, subset=sellected_features)

# Data Structure of Rondom forest

In [7]:
class Node:                                                                  
    def __init__(self, left=None, right=None, feature_name=None, threshold=None, predicted_value=None, data_cnt=None):
        # Only be used at internal node                                      
        self.left = left                                                     
        self.right = right                                                   
        self.feature_name = feature_name                                     
        self.threshold = threshold                                           
                                                                             
        # Only be used at leaf node                                          
        self.predicted_value = predicted_value                               
                                                                             
        # For debug                                                          
        self.data_cnt = data_cnt                                             
                                                                             
# class of DecisionTree                                                      
class DecisionTree:                                                          
    def __init__(self, max_depth, target_name, iter_feature_cnt, min_sample_cnt, loss_type):
        self.root = None                                                     
        self.max_depth = max_depth                                           
        self.target_name = target_name                                       
                                                                             
        # How many features will be choosed in each iteration during new node creation
        self.iter_feature_cnt = iter_feature_cnt                             
                                                                             
        # Used for randomly pick features                                    
        self.dice = None                                                     
        self.feature_list = None                                             
                                                                             
        # If the sample count is less than min_sample_cnt, stop creating new node.
        self.min_sample_cnt = min_sample_cnt                                 
        self.loss_type = loss_type                                           
                                                                             
    def fit(self, dataset):                                                  
        self.tree_init(dataset)                                              
        self.root = self.create_node(dataset=dataset, curr_depth=1)          
                                                                             
    def tree_init(self, dataset):                                            
        self.feature_list = list(dataset.columns)                            
        self.feature_list.remove(self.target_name)                           
        self.dice = range(len(self.feature_list))                            
                                                                             
    def create_node(self, dataset, curr_depth):                              
        # Use recursion to create the whole decision tree                    
                                                                             
        # If the following two items is True, return a node with left & right child node.
        # 1. Current depth less than max depth                               
        # 2. Data count larger than minimum sample count                     
        if curr_depth <= self.max_depth and len(dataset) >= self.min_sample_cnt:
            feature_name, threshold, left_dataset, right_dataset = self.best_split(dataset)
                                                                             
            # Create internal node                                           
            return Node(left=self.create_node(left_dataset, curr_depth+1),   
                        right=self.create_node(right_dataset, curr_depth+1), 
                        feature_name=feature_name,                           
                        threshold=threshold,                                 
                        data_cnt=dataset.shape[0])                           
                                                                             
        # Otherwise, return a leaf node.                                     
        predicted_value = dataset[self.target_name].mean()                   
        return Node(predicted_value=predicted_value,                         
                    data_cnt=dataset.shape[0])                               
                                                                             
    def roll_dice(self):                                                     
        # Used for randomly choose features                                  
        # The size of dice is equal to features count                        
        self.dice = random.sample(self.dice, len(self.dice))                 
                                                                             
    def best_split(self, dataset):                                           
        min_loss = sys.float_info.max                                        
        best_feature_name = None                                             
        best_threshold = None                                                
                                                                             
        # Selcet feature                                                     
        self.roll_dice()                                                     
        for i in range(self.iter_feature_cnt):                               
            curr_feature_name = self.feature_list[self.dice[i]]              
                                                                             
            # Select threshold                                               
            unique_value_list = np.unique(dataset[curr_feature_name])        
            for idx in range(len(unique_value_list) - 1):                    
                curr_threshold = (unique_value_list[idx] + unique_value_list[idx+1]) / 2
                                                                             
                # Split & compute loss                                       
                left_dataset, right_dataset = self.split(dataset, curr_feature_name, curr_threshold)
                curr_loss = self.compute_loss(left_dataset[self.target_name], 
                                              right_dataset[self.target_name])
                                                                             
                if curr_loss < min_loss:                                     
                    best_left_dataset, best_right_dataset = left_dataset, right_dataset
                    best_feature_name = curr_feature_name                    
                    best_threshold = curr_threshold                          
                    min_loss = curr_loss                                     
                                                                             
        return best_feature_name, best_threshold, best_left_dataset, best_right_dataset
                                                                             
    def split(self, dataset, feature_name, threshold):                       
        left_dataset = dataset[dataset[feature_name] <= threshold]           
        right_dataset = dataset[dataset[feature_name] > threshold]           
        return left_dataset, right_dataset                                   
                                                                             
    def compute_loss(self, left_y, right_y):                                 
        if self.loss_type == 'MAE':                                          
            return self.MAE(left_y) + self.MAE(right_y)                      
        else:                                                                
            raise ValueError("Parameter 'loss_type': " + self.loss_type + " is invalid.")
                                                                             
    def MAE(self, y):                                                        
        return sum([(ele - sum(y)) ** 2 for ele in y])                       
                                                                             
    def predict_all(self, X):                                                
        data_cnt = len(X)                                                    
        y = [self.predict_single(X.iloc[i,:]) for i in range(data_cnt)]      
        return y                                                             
                                                                             
    def predict_single(self, x):                                             
        curr_node = self.root                                                
                                                                             
        while curr_node.predicted_value == None:                             
            if x[curr_node.feature_name] <= curr_node.threshold:             
                curr_node = curr_node.left                                   
            else:                                                            
                curr_node = curr_node.right                                  
        return curr_node.predicted_value                                     
                                                                             
    def print_prediction(self, y, pred_y):                                   
        for i in range(len(y)):                                              
            print('Observation: ' + str(y[i]), end='\t')                     
            print('Prediction: ' + str(list(pred_y)[i]))                     
                                                                             

# Data Structure of Decision Tree

In [8]:
class RandomForest:                                                                   
    def __init__(self, tree_cnt, max_depth, target_name, iter_feature_cnt, min_sample_cnt, loss_type):
        self.tree_cnt = tree_cnt                                                      
        self.max_depth = max_depth                                                    
        self.target_name = target_name                                                
        self.loss_type = loss_type                                                    
                                                                                      
        # How many features will be choosed in each iteration during new node creation
        self.iter_feature_cnt = iter_feature_cnt                                      
                                                                                      
        # If the sample count is less than min_sample_cnt, stop creating new node.    
        self.min_sample_cnt = min_sample_cnt                                          
                                                                                      
        self.tree_list = []                                                           
                                                                                      
    def fit(self, dataset):                                                           
        for i in range(tree_cnt):                                                     
            print('Fitting tree ' + str(i))                                           
                                                                                      
            # Randomly pick observations(bagging)                                     
            data_cnt = dataset.shape[0]                                               
            new_dataset = pd.DataFrame(columns=dataset.columns)                       
            for j in range(data_cnt):                                                 
                idx = random.randint(0, data_cnt-1)                                   
                new_dataset.loc[j] = list(dataset.iloc[idx,:])                        
                                                                                      
            # Fitting decision tree with randomly pick observations                   
            dt = DecisionTree(self.max_depth,                                         
                              self.target_name,                                       
                              self.iter_feature_cnt,                                  
                              self.min_sample_cnt,                                    
                              self.loss_type)                                         
            dt.fit(new_dataset)                                                       
            self.tree_list.append(dt)                                                 
                                                                                      
    def predict_all(self, X):                                                         
        curr_dt = self.tree_list[0]                                                   
        tree_cnt = len(self.tree_list)                                                
        pred_y = curr_dt.predict_all(X)                                               
                                                                                      
        for i in range(1, tree_cnt):                                                  
            curr_dt = self.tree_list[i]                                               
            pred_y_tmp = curr_dt.predict_all(X)                                       
            pred_y = [sum(ele) for ele in zip(pred_y, pred_y_tmp)]                    
                                                                                      
        pred_y = [ele / tree_cnt for ele in pred_y]                                   
        return pred_y                                                                 
                                                                                      
    def print_prediction(self, y, pred_y):                                            
        for i in range(len(y)):                                                       
            print('Observation: ' + str(y.iloc[i]), end='\t')                         
            print('Prediction: ' + str(list(pred_y)[i]), end='\t')                    
            print('MAE = ' + str((y.iloc[i] - list(pred_y)[i]) ** 2) )                

In [9]:
tree_cnt = 1              
max_depth = 1             
target_name = 'SalePrice' 
iter_feature_cnt = 1      
min_sample_cnt = 50       
loss_type = 'MAE'         
                          
rf = RandomForest(tree_cnt, max_depth, target_name, iter_feature_cnt, min_sample_cnt, loss_type)
rf.fit(X_full)            

Fitting tree 0


In [10]:
pred_y = rf.predict_all(X_test_full)                 
rf.print_prediction(X_test_full[target_name], pred_y)

Observation: 136000	Prediction: 151956.89199029127	MAE = 254622401.98982185
Observation: 287090	Prediction: 235094.08899676375	MAE = 2703574761.0564647
Observation: 145000	Prediction: 151956.89199029127	MAE = 48398346.16457889
Observation: 84500	Prediction: 151956.89199029127	MAE = 4550432276.989823
Observation: 185000	Prediction: 235094.08899676375	MAE = 2509417752.415687
Observation: 175000	Prediction: 151956.89199029127	MAE = 530984826.7471024
Observation: 210000	Prediction: 235094.08899676375	MAE = 629713302.5774994
Observation: 266500	Prediction: 235094.08899676375	MAE = 986331245.9431958
Observation: 142125	Prediction: 151956.89199029127	MAE = 96666100.10875373
Observation: 147500	Prediction: 235094.08899676375	MAE = 7672724427.172968


In [11]:
tree_cnt = 10                                                                  
max_depth = 1                                                                  
target_name = 'SalePrice'                                                      
iter_feature_cnt = 1                                                           
min_sample_cnt = 50                                                            
loss_type = 'MAE'                                                              
                                                                               
rf2 = RandomForest(tree_cnt, max_depth, target_name, iter_feature_cnt, min_sample_cnt, loss_type)
rf2.fit(X_full)                                                                

Fitting tree 0
Fitting tree 1
Fitting tree 2
Fitting tree 3
Fitting tree 4
Fitting tree 5
Fitting tree 6
Fitting tree 7
Fitting tree 8
Fitting tree 9


In [12]:
pred_2_y = rf2.predict_all(X_test_full)                 
rf2.print_prediction(X_test_full[target_name], pred_2_y)

Observation: 136000	Prediction: 158893.0303532661	MAE = 524090838.7555632
Observation: 287090	Prediction: 225416.27516896892	MAE = 3803648334.53374
Observation: 145000	Prediction: 169604.85295299167	MAE = 605398788.8383429
Observation: 84500	Prediction: 149709.11278700645	MAE = 4252228390.4685283
Observation: 185000	Prediction: 199872.4635612333	MAE = 221190172.38021252
Observation: 175000	Prediction: 180779.55189083423	MAE = 33403220.058845576
Observation: 210000	Prediction: 185644.09274720986	MAE = 593210218.1065155
Observation: 266500	Prediction: 205520.5350029837	MAE = 3718495151.322336
Observation: 142125	Prediction: 143041.61698389036	MAE = 840186.6951562583
Observation: 147500	Prediction: 143041.61698389036	MAE = 19877179.1183349
